In [20]:
import mysql.connector

# إنشاء اتصال بقاعدة البيانات MySQL
connection_mydb = mysql.connector.connect(
    host='localhost',   # اسم المضيف (عادةً localhost عند تشغيل قاعدة البيانات محليًا)
    user='root',        # اسم المستخدم لقاعدة البيانات
    password='',        # كلمة المرور لقاعدة البيانات (تُركت فارغة هنا)
    database='wp-ecommerce'  # اسم قاعدة البيانات المراد الاتصال بها
)

In [21]:
def get_product_name_from_id(connection_mydb, product_id):

    # إنشاء كائن المؤشر للتعامل مع قاعدة البيانات
    cursor = connection_mydb.cursor(dictionary=True)

    # استعلام SQL للحصول على اسم المنتج بناءً على معرفه
    sql = "SELECT post_title FROM wp_posts WHERE ID=(%s)"

    # تمرير معرف المنتج كمعامل للاستعلام
    id = (product_id,)
    cursor.execute(sql, id)

    # جلب النتائج من قاعدة البيانات
    results = cursor.fetchall()

    # التحقق مما إذا كان المنتج موجودًا وإرجاع اسمه
    if len(results) > 0:
        return results[0]['post_title']
        
    # إرجاع اسم افتراضي في حال لم يتم العثور على المنتج
    return "Unknown Product"

# اختبار الدالة باستخدام معرف منتج معين
get_product_name_from_id(connection_mydb, 55945)

"بنطال قماش رجالي-أزرق-Levi's"

In [22]:
# اختبار الدالة باستخدام معرف منتج معين
get_product_name_from_id(connection_mydb, 99999)

'Unknown Product'

In [19]:
import pandas as pd

def build_dataframe_associated_products(connection_mydb):
    
    # إنشاء DataFrame فارغ يحتوي على 10 أعمدة لتخزين بيانات المنتجات المرتبطة
    df = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9])
    
    # إنشاء كائن cursor لتنفيذ الاستعلامات على قاعدة البيانات
    cursor = connection_mydb.cursor(dictionary=True)
    
    # جلب جميع الطلبات من جدول الطلبات وترتيبها حسب رقم الطلب
    sql = "SELECT * FROM wp_wc_order_stats ORDER BY order_id"
    cursor.execute(sql)
    results_orders = cursor.fetchall()
    
    # المرور على كل طلب في قاعدة البيانات
    for order in results_orders:
        order_id = order['order_id']
        
        # جلب جميع المنتجات المرتبطة بالطلب الحالي
        sql = "SELECT * FROM wp_wc_order_product_lookup WHERE order_id = (%s)"
        id = (order_id,)
        cursor.execute(sql, id)
        
        results_products = cursor.fetchall()
        
        # قائمة لتخزين معرفات المنتجات المرتبطة بنفس الطلب
        products_ids = []
        for product in results_products:
            product_id = product['product_id']
            
            # التأكد من أن معرف المنتج صالح قبل إضافته للقائمة
            if product_id > 0:
                products_ids.append(product_id)
        
        # إضافة البيانات إلى DataFrame إذا كان الطلب يحتوي على أكثر من منتج واحد
        if len(products_ids) > 1:
            df = pd.concat([df, pd.DataFrame([products_ids])], ignore_index=True)
    
    return df

# تنفيذ الدالة لجلب بيانات المنتجات المرتبطة
df = build_dataframe_associated_products(connection_mydb)
df

,0,1,2,3,4,5,6,7,8,9
0,55959,55979,56265,55968,NaN,NaN,NaN,NaN,NaN,NaN
1,55945,55947,56108,56261,55973,NaN,NaN,NaN,NaN,NaN
2,55890,55931,56064,56297,55898,NaN,NaN,NaN,NaN,NaN
3,56007,55982,56152,56286,55981,NaN,NaN,NaN,NaN,NaN
4,55884,55886,56082,56288,55879,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2540,56214,56247,56238,56235,NaN,NaN,NaN,NaN,NaN,NaN
2541,56354,56341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2542,55958,56005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2543,56007,56115,55976,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
def prepare_transactions(df):

    # تحويل الجدول بحيث تصبح الطلبات في الصفوف والعناصر في الأعمدة
    df = df.T

    # تحويل كل صف إلى قائمة من العناصر غير الفارغة (التعامل مع القيم الفارغة)
    transactions = df.apply(lambda x: x.dropna().tolist())

    # تحويل الأعمدة إلى قائمة من القوائم لتمثل المعاملات
    transactions_list = transactions.values.tolist()

    # استيراد TransactionEncoder لترميز البيانات
    from mlxtend.preprocessing import TransactionEncoder

    te = TransactionEncoder()

    # تحويل البيانات إلى مصفوفة ثنائية حيث يمثل كل صف معاملة وكل عمود منتج
    rows = te.fit_transform(transactions_list)

    # إنشاء إطار بيانات من المصفوفة المشفرة
    df_transactions = pd.DataFrame(rows, columns=te.columns_)

    return df_transactions

# استدعاء الدالة وتحضير المعاملات
df_transactions = prepare_transactions(df)

# عرض إطار البيانات الناتج
df_transactions

,55868,55869,55870,55873,55874,55876,55877,55878,55879,55881,...,56345,56347,56350,56351,56354,56355,56358,56359,56360,56363
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2541,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2542,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2543,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [28]:
def generate_association_rules(df_transactions, min_support, min_confidence):

    # استيراد دالة Apriori لاستخراج العناصر المتكررة
    from mlxtend.frequent_patterns import apriori

    # استخراج مجموعات العناصر المتكررة بناءً على الحد الأدنى للدعم
    frequent_itemsets = apriori(df_transactions, min_support=min_support, use_colnames=True)

    # استيراد دالة استخراج قواعد الارتباط
    from mlxtend.frequent_patterns import association_rules

    # إنشاء قواعد الارتباط بناءً على الثقة المحددة
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

    # ترتيب القواعد بناءً على الثقة من الأعلى إلى الأدنى
    rules = rules.sort_values(['confidence'], ascending=[False])

    return rules

# استدعاء الدالة لتوليد القواعد مع تحديد القيم الدنيا للدعم والثقة
rules = generate_association_rules(df_transactions, 0.004, 0.1)

# عرض القواعد الناتجة
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
15,(56235),(56193),0.009430,0.031434,0.004322,0.458333,14.580729,1.0,0.004026,1.788122,0.940283,0.118280,0.440754,0.297917
28,(56209),(56238),0.012181,0.036149,0.005501,0.451613,12.492987,1.0,0.005061,1.757610,0.931299,0.128440,0.431046,0.301893
51,(56323),(56305),0.010609,0.016896,0.004322,0.407407,24.112834,1.0,0.004143,1.658988,0.968806,0.186441,0.397223,0.331611
4,(56179),(56238),0.017289,0.036149,0.006287,0.363636,10.059289,1.0,0.005662,1.514623,0.916433,0.133333,0.339769,0.268775
1,(56179),(56218),0.017289,0.029862,0.006287,0.363636,12.177033,1.0,0.005771,1.524502,0.934026,0.153846,0.344048,0.287081
20,(56195),(56238),0.015717,0.036149,0.005501,0.350000,9.682065,1.0,0.004933,1.482847,0.911035,0.118644,0.325622,0.251087
40,(56227),(56228),0.014931,0.031041,0.004715,0.315789,10.173218,1.0,0.004252,1.416170,0.915370,0.114286,0.293870,0.233844
8,(56188),(56228),0.017682,0.031041,0.005501,0.311111,10.022504,1.0,0.004952,1.406553,0.916429,0.127273,0.289042,0.244163
35,(56245),(56217),0.014145,0.033399,0.004322,0.305556,9.148693,1.0,0.003850,1.391906,0.903475,0.100000,0.281561,0.217484
42,(56232),(56228),0.015717,0.031041,0.004715,0.300000,9.664557,1.0,0.004227,1.384227,0.910845,0.112150,0.277575,0.225949


In [34]:
rules=generate_association_rules(df_transactions, 0.004, 0.1)
rules.shape

(52, 14)

In [35]:
rules = generate_association_rules(df_transactions, 0.01, 0.1)
rules.shape

(0, 14)

In [37]:
# تعيين القيم الأولية للدعم والثقة
support = 0.01  # الحد الأدنى لقيمة الدعم (support) التي سنبدأ بها
confidence = 0.1  # الحد الأدنى لقيمة الثقة (confidence)
n = 0  # متغير لحفظ عدد القواعد المستخرجة
steps = 0  # عداد لتتبع عدد الخطوات المتخذة

# حلقة تكرارية لضبط قيمة الدعم حتى نصل إلى 2000 قاعدة على الأقل
while n < 2000:
    support = support / 2  # تقليل قيمة الدعم إلى النصف في كل تكرار
    steps = steps + 1  # زيادة عدد الخطوات بمقدار 1
    
    # استدعاء دالة استخراج قواعد الارتباط مع القيم الجديدة للدعم والثقة
    rules = generate_association_rules(df_transactions, support, confidence)
    
    n = rules.shape[0]  # تحديث عدد القواعد المستخرجة
    
    # طباعة القيم الحالية لكل تكرار
    print(" rules= ", n, ", support= ", support, ", confidence= ", confidence)

 rules=  26 , support=  0.005 , confidence=  0.1
 rules=  184 , support=  0.0025 , confidence=  0.1
 rules=  535 , support=  0.00125 , confidence=  0.1
 rules=  2460 , support=  0.000625 , confidence=  0.1


In [38]:
def predict(rules, items, max_results=6):

    # تصفية القواعد حيث تكون المقدمة (antecedents) مطابقة للعناصر المدخلة
    preds = rules[rules['antecedents'] == items]

    # الاحتفاظ فقط بالأعمدة المتعلقة بالنتائج والتوثيق (الثقة)
    preds = preds[['consequents', 'confidence']]

    # فرز النتائج حسب نسبة الثقة من الأعلى إلى الأدنى
    preds.sort_values('confidence', ascending=False)

    # إرجاع عدد محدود من النتائج وفقًا لـ max_results
    return preds[:max_results]

In [39]:
preds = predict(rules, {56193})

preds

,consequents,confidence
1178,(56228),0.2000
1186,(56247),0.1625
1180,(56230),0.1500
1183,(56235),0.1375
1170,(56217),0.1250
1115,(56183),0.1125


In [41]:
# تعيين معرف المنتج المطلوب التوصية له
product_id = 56193

# استدعاء دالة التوقع بناءً على قواعد الارتباط
preds = predict(rules, {product_id})

# طباعة اسم المنتج الأساسي الذي نريد توصيات له
print("Item = ", get_product_name_from_id(connection_mydb, product_id))

# طباعة العنوان للتوصيات الناتجة
print("Recommandations : ")

# المرور على كل صف في نتائج التوقعات
for _, row in preds.iterrows():
    
    # المرور على كل معرف منتج في مجموعة النتائج الموصى بها
    for id in row['consequents']:
        
        # جلب اسم المنتج باستخدام معرفه من قاعدة البيانات
        name = get_product_name_from_id(connection_mydb, int(id))
        
        # طباعة معرف المنتج واسمه
        print(id, name)

Item =  فرن-بني-HP
Recommandations : 
56228 غسالة-أسود-LG
56247 غسالة-فضي-SONY
56230 غسالة-أسود-SONY
56235 مصفف شعر-أسود داكن-SONY
56217 غسالة-أسود-SAMSUNG
56183 براد-فضي-PANASONIC


In [ ]:
def export_to_db(connection_mydb, rules):
    # إنشاء كائن المؤشر للتفاعل مع قاعدة البيانات
    cursor = connection_mydb.cursor()
    
    # حذف الجدول إذا كان موجودًا مسبقًا
    sql = '''DROP TABLE IF EXISTS custom_products_association'''
    cursor.execute(sql)
    
    # إنشاء جدول جديد لتخزين بيانات ارتباط المنتجات
    sql = '''
    CREATE TABLE custom_products_association ( 
        ID int(11) NOT NULL AUTO_INCREMENT,
        product_id_in int(11) NOT NULL,
        post_title_in text NOT NULL,
        product_id_out int(11) NOT NULL,
        post_title_out text NOT NULL,
        confidence double NOT NULL, 
        PRIMARY KEY (ID)
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
    '''
    cursor.execute(sql)
    
    # تأكيد التغييرات في قاعدة البيانات
    connection_mydb.commit()
    
    # المرور عبر كل صف في DataFrame rules
    for row in rules.itertuples():
        antecedents = row.antecedents  # الحصول على المنتجات المدخلة
        consequents = row.consequents    # الحصول على المنتجات الخارجة
        confidence = row.confidence * 100  # تحويل الثقة إلى نسبة مئوية
        
        # المرور عبر كل منتج في المدخلات
        for product_in in antecedents:
            post_title_in = get_product_name_from_id(connection_mydb, product_in)  # الحصول على اسم المنتج المدخل
            
            # المرور عبر كل منتج في المخرجات
            for product_out in consequents:
                post_title_out = get_product_name_from_id(connection_mydb, product_out)  # الحصول على اسم المنتج الخارج
                
                # حذف السجلات التي تتطابق مع المنتج المدخل والخارج وثقة أقل أو تساوي القيمة الحالية
                sql = '''DELETE FROM custom_products_association WHERE product_id_in=(%s) AND
                product_id_out=(%s) AND confidence <= (%s) '''
                params = (product_in, product_out, confidence)
                cursor.execute(sql, params)
                
                # التحقق مما إذا كانت هناك سجلات موجودة مع نفس المنتج المدخل والخارج وثقة أكبر أو تساوي القيمة الحالية
                sql = '''SELECT * FROM custom_products_association WHERE product_id_in=(%s) AND
                product_id_out=(%s) AND confidence >= (%s) '''
                params = (product_in, product_out, confidence)
                cursor.execute(sql, params)
                results = cursor.fetchall()
                
                must_add = True  # افتراض أنه يجب إضافة السجل الجديد
                if results is not None and len(results) > 0:
                    must_add = False  # إذا كانت هناك سجلات موجودة، فلا حاجة للإضافة
                
                # إذا لم تكن هناك سجلات موجودة، قم بإضافة السجل الجديد
                if must_add:
                    sql = "INSERT INTO custom_products_association (product_id_in, post_title_in, product_id_out, post_title_out, confidence) VALUES (%s, %s, %s, %s, %s)"
                    val = (product_in, post_title_in, product_out, post_title_out, confidence)
                    cursor.execute(sql, val)
                    
                    # تأكيد التغييرات في قاعدة البيانات
                    connection_mydb.commit()

In [ ]:
def start_generate_association_rules():
    # إنشاء اتصال بقاعدة البيانات MySQL
    connection_mydb = mysql.connector.connect(
        host="localhost",  # عنوان الخادم
        user="root",       # اسم المستخدم
        password="",       # كلمة المرور (فارغة في هذه الحالة)
        database="wp-ecommerce"  # اسم قاعدة البيانات
    )

    # تحديد الحد الأدنى للدعم (min_support) لقاعدة البيانات
    min_support = 0.001

    # تحديد الحد الأدنى للثقة (min_confidence) لقاعدة البيانات
    min_confidence = 0.001

    # بناء DataFrame للمنتجات المرتبطة من قاعدة البيانات
    df = build_dataframe_associated_products(connection_mydb)

    # إعداد بيانات المعاملات من DataFrame
    transactions_df = prepare_transactions(df)

    # توليد قواعد الارتباط باستخدام بيانات المعاملات والحدود المحددة
    rules = generate_association_rules(transactions_df, min_support, min_confidence)

    # تصدير قواعد الارتباط إلى قاعدة البيانات
    export_to_db(connection_mydb, rules)

In [ ]:
start_generate_association_rules()